# Study 1 for leaf recognition

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
from sklearn.preprocessing import StandardScaler, Normalizer
import math
import matplotlib
import seaborn as sns
from sklearn import preprocessing
from scipy.stats import skew
from scipy.stats import boxcox
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from scipy.stats import boxcox



### TODO
1. ~~basic info~~
3. ~~nulls~~
2. skewness
3. outliers
3. important features and correlation matrix

In [2]:
df_train = pd.read_csv('train.csv.zip', sep=',', header=0, compression='zip',encoding='cp1252')
df_test =  pd.read_csv('test.csv.zip', sep=',', header=0, compression='zip',encoding='cp1252')
print(df_train.shape)
print(df_test.shape)


(990, 194)
(594, 193)


In [3]:
df_train.sample(5)

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
77,134,Quercus_Afares,0.017578,0.015625,0.046875,0.013672,0.005859,0.007812,0.007812,0.000000,...,0.000000,0.000000,0.001953,0.016602,0.006836,0.006836,0.0,0.018555,0.033203,0.022461
674,1076,Pterocarya_Stenoptera,0.001953,0.017578,0.003906,0.005859,0.029297,0.000000,0.021484,0.000000,...,0.000000,0.000000,0.000000,0.004883,0.039062,0.000000,0.0,0.008789,0.021484,0.000000
295,488,Betula_Austrosinensis,0.001953,0.000000,0.005859,0.015625,0.068359,0.000000,0.021484,0.000000,...,0.064453,0.001953,0.004883,0.007812,0.011719,0.000000,0.0,0.058594,0.005859,0.004883
426,698,Acer_Rubrum,0.000000,0.001953,0.015625,0.005859,0.033203,0.007812,0.005859,0.000000,...,0.001953,0.000000,0.002930,0.000000,0.000000,0.000000,0.0,0.039062,0.000000,0.001953
550,877,Alnus_Maximowiczii,0.000000,0.000000,0.007812,0.011719,0.039062,0.000000,0.003906,0.001953,...,0.000000,0.000977,0.000977,0.011719,0.013672,0.035156,0.0,0.080078,0.000000,0.018555


In [4]:
df_train.describe()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
count,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,...,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000
mean,799.595960,0.017412,0.028539,0.031988,0.023280,0.014264,0.038579,0.019202,0.001083,0.007167,...,0.036501,0.005024,0.015944,0.011586,0.016108,0.014017,0.002688,0.020291,0.008989,0.019420
std,452.477568,0.019739,0.038855,0.025847,0.028411,0.018390,0.052030,0.017511,0.002743,0.008933,...,0.063403,0.019321,0.023214,0.025040,0.015335,0.060151,0.011415,0.039040,0.013791,0.022768
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,415.250000,0.001953,0.001953,0.013672,0.005859,0.001953,0.000000,0.005859,0.000000,0.001953,...,0.000000,0.000000,0.000977,0.000000,0.004883,0.000000,0.000000,0.000000,0.000000,0.000977
50%,802.500000,0.009766,0.011719,0.025391,0.013672,0.007812,0.015625,0.015625,0.000000,0.005859,...,0.004883,0.000000,0.005859,0.000977,0.012695,0.000000,0.000000,0.003906,0.002930,0.011719
75%,1195.500000,0.025391,0.041016,0.044922,0.029297,0.017578,0.056153,0.029297,0.000000,0.007812,...,0.043701,0.000000,0.022217,0.009766,0.021484,0.000000,0.000000,0.023438,0.012695,0.029297
max,1584.000000,0.087891,0.205080,0.156250,0.169920,0.111330,0.310550,0.091797,0.031250,0.076172,...,0.429690,0.202150,0.172850,0.200200,0.106450,0.578130,0.151370,0.375980,0.086914,0.141600


In [5]:
df_train.isnull().sum().sum()

0

In [6]:
means = df_train.ix[:,2:].mean()
( means.max() - means.min(), means.std() )

(0.041141939393939467, 0.009711506745082836)

In [7]:
df_train.dtypes[df_train.dtypes!='float64']

id          int64
species    object
dtype: object

In [8]:
X = df_train.ix[:,2:]
Y = df_train.ix[:,1]

enc = preprocessing.LabelEncoder()
Y = enc.fit_transform(Y)

X_tr, X_val, y_tr, y_val = train_test_split(X, Y, test_size=.2, random_state = 13)
(X_tr.shape, X_val.shape)

((792, 192), (198, 192))

In [9]:
svc = svm.SVC()

param_grid = [
  {'C': [10, 50, 75, 100, 125, 250], 'kernel': ['linear'], 'tol': [0.001, 0.0001]},
 ]

gs = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
gs = gs.fit(X_tr, y_tr)

be = gs.best_estimator_
print("best score: " , gs.best_score_)
print("best params: " , gs.best_params_)
print(be)
print("Grid scores on development set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

svc = svm.SVC(C=be.C, kernel=be.kernel, probability=True).fit(X_tr, y_tr)
svc.score(X_val, y_val)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    4.0s finished


best score:  0.929292929293
best params:  {'kernel': 'linear', 'tol': 0.001, 'C': 100}
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Grid scores on development set:

0.811 (+/-0.061) for {'kernel': 'linear', 'tol': 0.001, 'C': 10}
0.811 (+/-0.061) for {'kernel': 'linear', 'tol': 0.0001, 'C': 10}
0.924 (+/-0.062) for {'kernel': 'linear', 'tol': 0.001, 'C': 50}
0.924 (+/-0.062) for {'kernel': 'linear', 'tol': 0.0001, 'C': 50}
0.927 (+/-0.062) for {'kernel': 'linear', 'tol': 0.001, 'C': 75}
0.927 (+/-0.062) for {'kernel': 'linear', 'tol': 0.0001, 'C': 75}
0.929 (+/-0.060) for {'kernel': 'linear', 'tol': 0.001, 'C': 100}
0.929 (+/-0.060) for {'kernel': 'linear', 'tol': 0.0001, 'C': 100}
0.928 (+/-0.058) for {'kernel': 'linear', 'tol': 0.001, 'C': 125}
0.928 (+/-0.058) for {'kernel': 'linear', 'tol': 0.0001, 'C'

0.9747474747474747

In [18]:
svc.fit(X, Y)

data = svc.predict_proba(df_test.ix[:,1:])
cols = enc.inverse_transform(svc.classes_)
index = df_test['id'].values

res = pd.DataFrame(data=data, columns=cols, index=index)

res.to_csv('submission_svc.csv', index_label='id')

In [12]:
# 1st attempt w/o unskewing: 2.49084
# 2nd 2.33773

In [23]:
params = {'C':[100, 1000], 'tol': [0.001, 0.0001]}
log_reg = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial')
clf = GridSearchCV(log_reg, params, scoring='log_loss', refit='True', n_jobs=-1, cv=5)
clf.fit(X_tr, y_tr)

print("best params: " + str(clf.best_params_))
for params, mean_score, scores in clf.grid_scores_:
  print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std(), params))
  print(scores)

log_reg = linear_model.LogisticRegression(solver='lbfgs'
                                          , multi_class='multinomial'
                                          , C=clf.best_params_['C']
                                         , tol=clf.best_params_['tol']).fit(X_tr, y_tr)
log_reg.score(X_val, y_val)



/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to

best params: {'tol': 0.001, 'C': 1000}
-0.898 (+/-0.039) for {'tol': 0.001, 'C': 100}
[-0.91249094 -0.87144493 -0.85257967 -0.91449729 -0.96523712]
-0.898 (+/-0.039) for {'tol': 0.0001, 'C': 100}
[-0.91249094 -0.87144493 -0.85257967 -0.91449729 -0.96523712]
-0.401 (+/-0.069) for {'tol': 0.001, 'C': 1000}
[-0.37532259 -0.36803355 -0.33118005 -0.47373414 -0.51341367]
-0.401 (+/-0.069) for {'tol': 0.0001, 'C': 1000}
[-0.37532259 -0.36803355 -0.33118005 -0.47373414 -0.51341367]


/opt/conda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:680: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


0.97979797979797978

In [24]:
log_reg.fit(X, Y)

data = log_reg.predict_proba(df_test.ix[:,1:])
cols = enc.inverse_transform(log_reg.classes_)
index = df_test['id'].values

res = pd.DataFrame(data=data, columns=cols, index=index)

res.to_csv('submission_lr.csv', index_label='id')